In [1]:
import os
import re
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.notebook import tqdm
from sklearn.preprocessing import StandardScaler
from matplotlib.ticker import FormatStrFormatter
from scipy.stats import ttest_1samp
import statsmodels.stats.api as sms
import pickle
%config Completer.use_jedi = False
%matplotlib widget

In [2]:
"""
Define I/O paths
"""
# input directores
inDIR_data = 'C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/data/'
inDIR_mods = 'C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/models/'

# output directories
outDIR_tables = 'C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/models/results_tables/'

# file name of gridded path intensity data (output from gps_to_gridded_path_intensity.ipynb)
griddata_f = '2017_grazing_time_gridded_all_for_model.csv'

# file name of model results
results_f = '2017_grazing_intensity_model_results.csv'

# file name of model coefficients
coefs_f = '2017_grazing_intensity_model_coefficients.csv'

In [3]:
"""
Define models for analyzing
"""
# create list of saved models
mod_list = ['M0', 'M1', 'M2_bm', 'M2_top', 'M3_pc', 'M3_top', 'M4_top', 'M_best']

# create dictionary to map top models to original models
top_mod_dict = {
    'M2_top': 'M2g',
    'M3_top': 'M3d',
    'M4_top': 'M4h',
    'M5_top': 'M5'
}

# create dictionaries of saved model results for AIC comparison
mod_aic_dict = {
    'TPC2': [top_mod_dict['M2_top'], 'M2_tpc'],
    'TPC3': [top_mod_dict['M3_top'], 'M3_tpc'],
    'TPC4': [top_mod_dict['M4_top'], top_mod_dict['M5_top']],
    'Hw1': ['M1', 'M5'],
    'Hw2': ['M2a', 'M2b', 'M2c', 'M2d', 'M2e', 'M2f', 'M2g'],
    'Hw3': ['M2a', 'M2b', 'M2c', 'M2d', 'M2e', 'M2f', 'M2g'],
    'Hw4a': ['M2a', 'M2b'],
    'Hw4b': ['M2f', 'M2g'],
    'Hw5': ['M2a', 'M2b', 'M2c', 'M2d', 'M2e', 'M2f', 'M2g'],
    'Hw6': ['M2a', 'M2b', 'M2c', 'M2d', 'M2e', 'M2f', 'M2g'],
    'Hw7': ['M3a', 'M3b', 'M3c', 'M3d'],
    'Hw8': ['M3a', 'M3b', 'M3c', 'M3d'],
    'Hw9': ['M3a', 'M3b', 'M3c', 'M3d'],
    'Ha1': ['M0', 'M1', 'M2', 'M3'],
    'Ha2': ['M0', 'M1', 'M2', 'M3', 'M4'],
    'Ha3': ['M0', 'M1', 'M2', 'M3', 'M4'],
    'Ha4': ['M0', 'M1', 'M2', 'M3', 'M4'],
}

# create dictionaries of saved scalers
scaler_dict = {}
for seas in ['early', 'mid', 'late']:
    scaler_dict[seas] = {}
    for c in ['dFence', 'dTank', 'Biomass', 'CP', 'PC_div']:
        with open(os.path.join(inDIR_mods, 'scaler_' + seas + '_' + c + '.pkl'), 'rb') as f:
            scaler_dict[seas][c] = pickle.load(f)

In [4]:
"""
Load data and models
"""
# read gridded data into dataframe
df_wkly_grid_full = pd.read_csv(os.path.join(inDIR_data, griddata_f), engine='python')

# read model results into dataframe
df_results = pd.read_csv(os.path.join(inDIR_mods, results_f), engine='python')

# read model results into dataframe
df_coefs = pd.read_csv(os.path.join(inDIR_mods, coefs_f), engine='python')

# create dictionary of the seasonal models
seas_mod_dict = {}
for seas in df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'train']['season'].unique():
    seas_mod_dict[seas] = {}
    for mod in mod_list:
        f_matched = [f for f in os.listdir(os.path.join(inDIR_mods, 'seas')) if mod in f and seas in f]
        if len(f_matched) == 1:
            seas_mod_dict[seas][mod] = sm.load(os.path.join(
                inDIR_mods, 'seas', f_matched[0]))
        else:
            print('ERROR: None and/or multiple matching files found:')
            display(f_matched)
            
# create dictionary of the pasture-specific models
past_mod_dict = {}
for past in df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'train']['Pasture'].unique():
    past_mod_dict[past] = {}
    for seas in df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'train']['season'].unique():
        past_mod_dict[past][seas] = {}
        for mod in mod_list:
            f_matched = [f for f in os.listdir(os.path.join(inDIR_mods, 'past')) if past in f and mod in f and seas in f]
            if len(f_matched) == 1:
                past_mod_dict[past][seas][mod] = sm.load(os.path.join(
                    inDIR_mods, 'past', f_matched[0]))
            else:
                print('ERROR: None and/or multiple matching files found:')
                display(f_matched)
        
# scale data back to original values
for c in ['dFence', 'dTank', 'Biomass', 'CP', 'PC_div']:
    for seas in ['early', 'mid', 'late']:
        scaler = scaler_dict[seas][c]
        df_wkly_grid_full.loc[df_wkly_grid_full['season'] == seas, c + '_orig'] = scaler.inverse_transform(
            df_wkly_grid_full.loc[df_wkly_grid_full['season'] == seas, c].values.reshape(-1, 1)).flatten()


In [6]:
"""
Predict grazing intensity for chosen model
"""
# Change rare classes to 'Other' to match training data
df_wkly_grid_full['PC_dmt'].replace({'Bare_veg': 'Other', 'Bare': 'Other', 'UNK': 'Other', 'Shrub': 'Other'}, 
                               inplace=True)

# set the model for prediction
pred_model = 'M_best'

# predict relative probability of grazing across all data from seasonal models
for seas, df_sub in df_wkly_grid_full.groupby('season'):
    df_wkly_grid_full.loc[df_wkly_grid_full['season'] == seas, 'pred_seas'] = seas_mod_dict[seas][pred_model].predict(df_sub) * df_wkly_grid_full['grazing_wkly_sum'] / (df_wkly_grid_full['grazing_wkly_sum'] / 
                                                                             df_wkly_grid_full.groupby(
                                                                                 ['Pasture', 'Steer_ID', 'week'])['grazing_secs'].transform('count'))

In [7]:
"""
Bin seasonal predictions for categorical plotting and mapping
"""
rel_freq_bins = [0.0, 0.6, 0.8, 1.2, 1.4, 1000.0]
bin_labs = ['Low', 'Mod. Low', 'Avg.', 'Mod. Hi', 'Hi']
df_wkly_grid_full['grazing_rel_freq_grp'] = pd.cut(df_wkly_grid_full['grazing_rel_freq'], 
                                          bins=rel_freq_bins, 
                                          labels=bin_labs,
                                          include_lowest=True)
df_wkly_grid_full['pred_seas_grp'] = pd.cut(df_wkly_grid_full['pred_seas'], 
                                          bins=rel_freq_bins, 
                                          labels=bin_labs,
                                          include_lowest=True)

In [8]:
"""
Create and organize subsets of gridded data for plotting and analyses
"""
df_wkly_grid_full['season'] = pd.Categorical(df_wkly_grid_full['season'], ['early', 'mid', 'late'])
df_wkly_grid_train = df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'train']
df_wkly_grid_test = df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'test']
df_wkly_grid_train_means = df_wkly_grid_train.groupby(
    ['season', 'week', 'Pasture', 'UTM_X', 'UTM_Y']).mean()#.reset_index()
for colname in ['PC_dmt', 'TPC_c']:
    df_wkly_grid_train_means[colname] = df_wkly_grid_train.groupby(
        ['season', 'week', 'Pasture', 'UTM_X', 'UTM_Y'])[colname].first()
df_wkly_grid_train_means = df_wkly_grid_train_means.reset_index().dropna()

In [9]:
"""
Plot observed versus predicted relative grazing intensity
"""
fig, ax = plt.subplots(figsize=(10, 6))
sns.pointplot(x='pred_seas_grp', y='grazing_rel_freq', hue='mod_data', hue_order=['train', 'test'],
              data=df_wkly_grid_full, ax=ax)
ax.axhline(y=1.0, linestyle='dashed', c='gray')
ax.set(xlabel='Predicted relative grazing intensity (binned)',
      ylabel='Observed relative grazing intensity')
leg = ax.legend()
# change legend texts
new_title = 'Dataset'
leg.set_title(new_title, prop={'weight': 'bold',
        'size': 12,
        })
new_labels = ['Training', 'Testing']
for t, l in zip(leg.texts, new_labels):
    t.set_text(l)
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/Best_model_fit.png', 
           bbox_inches='tight', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [160]:
#g = sns.FacetGrid(data=df_wkly_grid_full, col='season', hue='mod_data')
#g.map_dataframe(sns.kdeplot, x='grazing_rel_freq', cumulative=True)
plt.figure()
sns.violinplot(data=df_wkly_grid_full, x='season', y='grazing_rel_freq', hue='mod_data', cut=0)
plt.ylim((0, 2))

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(0.0, 2.0)

In [161]:
g = sns.catplot(data=df_wkly_grid_full, x='pred_seas_grp', y='grazing_rel_freq', col='season', kind='point',
                hue='mod_data', hue_order=['train', 'test'], 
                  legend=False, sharey=True, height=4, aspect=0.8).add_legend(bbox_to_anchor=(0.5, 0.98), 
                                                  loc='upper center',
                                                  borderaxespad=0,
                                                  ncol=2,
                                                 frameon=True)
g.set(xlabel='',
     ylabel='')
g.fig.supxlabel('Predicted relative grazing intensity (binned)', y=-0.05)
g.fig.supylabel('Observed relative grazing intensity', x=-0.01)
for ax in g.axes.flat:
    ax.axhline(y=1.0, linestyle='dashed')
    ax.set_ylim((0.5, 1.5))
    cur_title = ax.get_title()
    ax.set_title(re.sub('season = ', '', cur_title + '-season'))
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width,box.height*0.80])
leg = g.legend
# change legend texts
new_title = 'Dataset'
leg.set_title(new_title, prop={'weight': 'bold',
        'size': 12,
        })
new_labels = ['Training', 'Testing']
for t, l in zip(leg.texts, new_labels):
    t.set_text(l)

plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/Best_model_fit_X_season.png', 
           bbox_inches='tight', dpi=300, pad_inches=0.1)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [92]:
"""
Get DataFrame of coefficients for best models for each season
"""
df_mbest_coefs_early = pd.read_html((seas_mod_dict['early']['M_best'].summary().tables[1]).as_html(), header=0, index_col=0)[0]
df_mbest_coefs_mid = pd.read_html((seas_mod_dict['mid']['M_best'].summary().tables[1]).as_html(), header=0, index_col=0)[0]
df_mbest_coefs_late = pd.read_html((seas_mod_dict['late']['M_best'].summary().tables[1]).as_html(), header=0, index_col=0)[0]

In [93]:
"""
Display best model coefficients for chosen season. Significant if red
"""

def style_sig(v, props=''):
    return np.repeat(props, len(v)) if v['P>|z|'] < 0.05 else None

print('\n---------------------- Early --------------------')
display(df_mbest_coefs_early.style.apply(style_sig, props='color:red;', axis=1))

print('\n---------------------- Mid --------------------')
display(df_mbest_coefs_mid.style.apply(style_sig, props='color:red;', axis=1))

print('\n---------------------- Late --------------------')
display(df_mbest_coefs_late.style.apply(style_sig, props='color:red;', axis=1))



---------------------- Early --------------------


,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-7.516800,0.033000,-225.674000,0.000000,-7.582000,-7.452000
"C(PC_dmt, Treatment(reference=""C3_C4_mix""))[T.C3]",1.856200,0.058000,32.177000,0.000000,1.743000,1.969000
"C(PC_dmt, Treatment(reference=""C3_C4_mix""))[T.C4]",-0.297900,0.038000,-7.869000,0.000000,-0.372000,-0.224000
"C(PC_dmt, Treatment(reference=""C3_C4_mix""))[T.Other]",-0.362000,0.167000,-2.169000,0.030000,-0.689000,-0.035000
"C(PC_dmt, Treatment(reference=""C3_C4_mix""))[T.Saltgrass]",0.538100,0.059000,9.094000,0.000000,0.422000,0.654000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Highlands]",-0.012600,0.043000,-0.291000,0.771000,-0.097000,0.072000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Lowlands]",0.447200,0.038000,11.633000,0.000000,0.372000,0.523000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Open Slopes]",0.074200,0.037000,1.991000,0.047000,0.001000,0.147000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Other]",-0.059300,0.190000,-0.312000,0.755000,-0.432000,0.314000
dFence,0.009100,0.014000,0.667000,0.505000,-0.018000,0.036000



---------------------- Mid --------------------


,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-7.237400,0.032000,-227.073000,0.000000,-7.300000,-7.175000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Highlands]",-0.254200,0.054000,-4.745000,0.000000,-0.359000,-0.149000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Lowlands]",0.120800,0.048000,2.500000,0.012000,0.026000,0.216000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Open Slopes]",-0.086900,0.048000,-1.830000,0.067000,-0.180000,0.006000
"C(TPC_c, Treatment(reference=""Flat Plains""))[T.Other]",-0.346800,0.243000,-1.425000,0.154000,-0.824000,0.130000
dFence,-0.016500,0.018000,-0.924000,0.355000,-0.051000,0.018000
dTank,0.022600,0.019000,1.200000,0.230000,-0.014000,0.060000
Biomass,-0.531400,0.024000,-22.526000,0.000000,-0.578000,-0.485000
I(Biomass ** 2),-0.179600,0.009000,-20.491000,0.000000,-0.197000,-0.162000
CP,0.966100,0.023000,41.364000,0.000000,0.920000,1.012000



---------------------- Late --------------------


,coef,std err,z,P>|z|,[0.025,0.975]
Intercept,-7.271500,0.050000,-145.491000,0.000000,-7.369000,-7.174000
dFence,0.065400,0.022000,2.951000,0.003000,0.022000,0.109000
dTank,-0.020500,0.020000,-1.025000,0.305000,-0.060000,0.019000
Biomass,-0.472200,0.026000,-18.328000,0.000000,-0.523000,-0.422000
I(Biomass ** 2),-0.350400,0.009000,-37.763000,0.000000,-0.369000,-0.332000
CP,0.449900,0.084000,5.374000,0.000000,0.286000,0.614000
PC_div,0.021400,0.022000,0.993000,0.321000,-0.021000,0.064000
alpha,12.053000,0.130000,92.585000,0.000000,11.798000,12.308000


In [94]:
"""
Reformat and save coefficients for table
"""
df_mbest_coefs_early = df_mbest_coefs_early.rename_axis('Param').reset_index()
df_mbest_coefs_early['Param'] = df_mbest_coefs_early['Param'].apply(lambda x: re.sub('\]', 
                                                                                     '',
                                                                                     re.sub('C\(PC_dmt, Treatment\(reference\="C3_C4_mix"\)\)\[T\.', 
                                                                                            '', 
                                                                                            str(x))))
df_mbest_coefs_early['Param'] = df_mbest_coefs_early['Param'].apply(lambda x: re.sub('\]', 
                                                                                     '',
                                                                                     re.sub('C\(TPC_c, Treatment\(reference\="Flat Plains"\)\)\[T\.', 
                                                                                            '', 
                                                                                            str(x))))

df_mbest_coefs_mid = df_mbest_coefs_mid.rename_axis('Param').reset_index()
df_mbest_coefs_mid['Param'] = df_mbest_coefs_mid['Param'].apply(lambda x: re.sub('\]', 
                                                                                     '',
                                                                                     re.sub('C\(PC_dmt, Treatment\(reference\="C3_C4_mix"\)\)\[T\.', 
                                                                                            '', 
                                                                                            str(x))))
df_mbest_coefs_mid['Param'] = df_mbest_coefs_mid['Param'].apply(lambda x: re.sub('\]', 
                                                                                     '',
                                                                                     re.sub('C\(TPC_c, Treatment\(reference\="Flat Plains"\)\)\[T\.', 
                                                                                            '', 
                                                                                            str(x))))

df_mbest_coefs_late = df_mbest_coefs_late.rename_axis('Param').reset_index()
df_mbest_coefs_late['Param'] = df_mbest_coefs_late['Param'].apply(lambda x: re.sub('\]', 
                                                                                     '',
                                                                                     re.sub('C\(PC_dmt, Treatment\(reference\="C3_C4_mix"\)\)\[T\.', 
                                                                                            '', 
                                                                                            str(x))))
df_mbest_coefs_late['Param'] = df_mbest_coefs_late['Param'].apply(lambda x: re.sub('\]', 
                                                                                     '',
                                                                                     re.sub('C\(TPC_c, Treatment\(reference\="Flat Plains"\)\)\[T\.', 
                                                                                            '', 
                                                                                            str(x))))
df_mbest_coefs_early.to_csv(os.path.join(outDIR_tables, 'M_best_coefs_early.csv'))
df_mbest_coefs_mid.to_csv(os.path.join(outDIR_tables, 'M_best_coefs_mid.csv'))
df_mbest_coefs_late.to_csv(os.path.join(outDIR_tables, 'M_best_coefs_late.csv'))

In [199]:
"""
FIGURE: Varibility in crude protein (a) and biomass (b) by sub-season across individual pastures 
"""
fig, axs = plt.subplots(nrows=2, figsize=(10, 6), sharex=True, gridspec_kw={'right': 0.8})
axs[0].axhline(y=8.0, linestyle='dashed', color='gray', alpha=0.8)
p1 = sns.violinplot(x='season', y='CP_orig', hue='Pasture', bw=0.5, cut=0,
            data=df_wkly_grid_train_means, ax=axs[0])
axs[0].set_ylabel('Crude protein (%)')
axs[0].set_xlabel(None)
axs[0]
handles, labels = axs[0].get_legend_handles_labels()
p1.legend().remove()
axs[1].axhline(y=500, linestyle='dashed', color='gray', alpha=0.8)
p2 = sns.violinplot(x='season', y='Biomass_orig', hue='Pasture', bw=0.5, cut=0,
            data=df_wkly_grid_train_means, ax=axs[1])
axs[1].set_ylabel('Biomass (kg/ha)')
axs[1].set_xlabel('Sub-season')
p2.legend().remove()
fig.legend(handles, labels, loc='center right', title='Pasture')
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/violinplots_CP_BM_X_Season_X_Pasture.png', 
            bbox_inches='tight', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [197]:
"""
FIGURE: Varibility in crude protein (a) and biomass (b) by sub-season across plant communities
"""
fig, axs = plt.subplots(nrows=2, figsize=(10, 6), sharex=True, gridspec_kw={'right': 0.8})
axs[0].axhline(y=8.0, linestyle='dashed', color='gray', alpha=0.8)
p1 = sns.violinplot(x='season', y='CP_orig', hue='PC_dmt', bw=0.5, cut=0,
            data=df_wkly_grid_train_means, ax=axs[0])
axs[0].set_ylabel('Crude protein (%)')
axs[0].set_xlabel(None)
handles, labels = axs[0].get_legend_handles_labels()
p1.legend().remove()
axs[1].axhline(y=500, linestyle='dashed', color='gray', alpha=0.8)
p2 = sns.violinplot(x='season', y='Biomass_orig', hue='PC_dmt', bw=0.5, cut=0,
            data=df_wkly_grid_train_means, ax=axs[1])
axs[1].set_ylabel('Biomass (kg/ha)')
axs[1].set_xlabel('Sub-season')
axs[1]
p2.legend().remove()
fig.legend(handles, labels, loc='center right', title='Plant community')
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/violinplots_CP_BM_X_Season_X_PCdmt.png', 
            bbox_inches='tight', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [198]:
"""
FIGURE: Varibility in crude protein (a) and biomass (b) by sub-season across plant communities
"""
fig, axs = plt.subplots(nrows=2, figsize=(10, 6), sharex=True, gridspec_kw={'right': 0.8})
axs[0].axhline(y=8.0, linestyle='dashed', color='gray', alpha=0.8)
p1 = sns.violinplot(x='season', y='CP_orig', hue='TPC_c', bw=0.5, cut=0,
            data=df_wkly_grid_train_means, ax=axs[0])
axs[0].set_ylabel('Crude protein (%)')
axs[0].set_xlabel(None)
handles, labels = axs[0].get_legend_handles_labels()
p1.legend().remove()
axs[1].axhline(y=500, linestyle='dashed', color='gray', alpha=0.8)
p2 = sns.violinplot(x='season', y='Biomass_orig', hue='TPC_c', bw=0.5, cut=0,
            data=df_wkly_grid_train_means, ax=axs[1])
axs[1].set_ylabel('Biomass (kg/ha)')
axs[1].set_xlabel('Sub-season')
axs[1]
p2.legend().remove()
fig.legend(handles, labels, loc='center right', title='Plant community')
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/violinplots_CP_BM_X_Season_X_TPC.png', 
            bbox_inches='tight', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [116]:
"""
Create dataframe of plant community coefficients by model and season
"""

pd.set_option('display.max_rows', 100)
df_coefs['Season'] = pd.Categorical(df_coefs['Season'], ['early', 'mid', 'late'])
coefs_PC_seas = df_coefs[df_coefs['Param'].str.contains('C\(PC_dmt\, Treatment\(reference\="C3_C4_mix"\)\)\[.*\]$', regex=True)]
coefs_PC_seas['Param'] = coefs_PC_seas['Param'].apply(lambda x: re.sub('C\(PC_dmt, Treatment\(reference\="C3_C4_mix"\)\)\[T\.', '', x))
coefs_PC_seas['Param'] = coefs_PC_seas['Param'].apply(lambda x: re.sub('\]', '', x))
coefs_PC_seas_agg = coefs_PC_seas.groupby(['Model','week', 'Season',  'Pasture', 'Param']).mean().reset_index().dropna().groupby(['Model', 'Season', 'Param'])['coef'].agg(
    [('coef_mean', np.mean),
     ('coef_lwr95', lambda x: sms.DescrStatsW(x).zconfint_mean()[0]),
     ('coef_upr95', lambda x: sms.DescrStatsW(x).zconfint_mean()[1]),
     ('pval', lambda x: np.round(ttest_1samp(x, 0.0)[1], 4))])#.reset_index()
#coefs_PC_seas_agg[coefs_PC_seas_agg.index.isin(['M3_pc', 'M_best'], level='Model')].dropna()

C:\Users\sean.kearney\.conda\envs\py_geo_env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [144]:
"""
Figure: Plant community coefficients and 95% CI by sub-season and by pasture for the plant-community-only model (M3_pc) and best model (M5)
"""
import matplotlib.ticker as ticker
g = sns.FacetGrid(coefs_PC_seas[coefs_PC_seas['Model'].isin(['M3_pc', 'M_best'])],
                  col='Season', legend_out=True, sharey=True, gridspec_kws={"wspace":0.4})
#plt.figure()
g.map_dataframe(sns.pointplot, x='coef', y='Param', hue='Model',
                join=False, dodge=0.3).add_legend(loc='upper center',
                                                  borderaxespad=0,
                                                  ncol=2,
                                                  frameon=True)
axs = g.axes
for ax in axs.flat:
    ax.collections[0]
    ax.set_xlim((-1.5, 1.5))
    ax.axvline(x=0, linestyle='dotted', color='black')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width,box.height*0.75])
    cur_title = ax.get_title()
    ax.set_title(re.sub('Season = ', '', cur_title + '-season'))
    ax.xaxis.set_major_formatter(ticker.FuncFormatter(lambda x, pos: '{:,.1f}'.format(x)))
    
leg = g.legend
# change legend texts
new_title = 'Model type'
leg.set_title(new_title, prop={'weight': 'bold',
        'size': 12,
        })
new_labels = ['Plant community only (M3a)', 'Fully paramaterized (M5)']
for t, l in zip(leg.texts, new_labels):
    t.set_text(l)
g.set_xlabels('Coefficient')
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/PC_dmt.png', 
            bbox_inches='tight', dpi=300)
#sns.pointplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

C:\Users\sean.kearney\.conda\envs\py_geo_env\lib\site-packages\seaborn\axisgrid.py:64: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  self.fig.tight_layout(*args, **kwargs)


In [145]:
"""
Create dataframe of topographic position class coefficients by model and season
"""
coefs_TPC_seas = df_coefs[df_coefs['Param'].str.contains('C\(TPC_c\, Treatment\(reference\="Flat Plains"\)\)\[.*\]$', regex=True)]
coefs_TPC_seas['Param'] = coefs_TPC_seas['Param'].apply(lambda x: re.sub('C\(TPC_c, Treatment\(reference\="Flat Plains"\)\)\[T\.', '', x))
coefs_TPC_seas['Param'] = coefs_TPC_seas['Param'].apply(lambda x: re.sub('\]', '', x))
coefs_TPC_seas_agg = coefs_TPC_seas.groupby(['Model','week', 'Season',  'Pasture', 'Param']).mean().reset_index().dropna().groupby(['Model', 'Season', 'Param'])['coef'].agg(
    [('coef_mean', np.mean),
     ('coef_lwr95', lambda x: sms.DescrStatsW(x).zconfint_mean()[0]),
     ('coef_upr95', lambda x: sms.DescrStatsW(x).zconfint_mean()[1]),
     ('pval', lambda x: np.round(ttest_1samp(x, 0.0)[1], 4))])#.reset_index()
#coefs_TPC_seas_agg[coefs_TPC_seas_agg.index.isin(['M1', 'M_best'], level='Model')].dropna()

C:\Users\sean.kearney\.conda\envs\py_geo_env\lib\site-packages\pandas\core\frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [146]:
"""
Figure: TPC coefficients and 95% CI by season and by pasture for the topographic position-only model (M1) and the best model (M5)
"""
g = sns.FacetGrid(coefs_TPC_seas[coefs_TPC_seas['Model'].isin(['M1', 'M_best'])],
                  col='Season', legend_out=True, sharey=True)
#plt.figure()
g.map_dataframe(sns.pointplot, x='coef', y='Param', hue='Model', join=False, dodge=0.3).add_legend(loc='upper center',
                                                                                                   borderaxespad=0, 
                                                                                                   ncol=2, 
                                                                                                   frameon=True)
axs = g.axes
for ax in axs.flat:
    ax.set_xlim((-1.5, 1.5))
    ax.axvline(x=0, linestyle='dotted', color='black')
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width, box.height*0.75])
    cur_title = ax.get_title()
    ax.set_title(re.sub('Season = ', '', cur_title + '-season'))
    
leg = g.legend
# change legend texts
new_title = 'Model type'
leg.set_title(new_title, prop={'weight': 'bold',
        'size': 12,
        })
new_labels = ['Topographic position only (M1)', 'Fully paramaterized (M5)']
for t, l in zip(leg.texts, new_labels):
    t.set_text(l)
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/TCP.png', 
            bbox_inches='tight', dpi=300)
#sns.pointplot()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [127]:
"""
Compare models by season: how does selection change with biomass and relative CP (within season)
"""
cp_q_dict = {
    'Lowest': 0.10,
    'Below avg': 0.25,
    'Average': 0.5,
    'Above avg': 0.75,
    'Highest': 0.90
}
ref_TPC = 'Flat Plains'
ref_pc = 'C3_C4_mix'
df_pred = pd.DataFrame(columns=['season','dFence_orig', 'dTank_orig', 'Biomass_orig', 'CP_orig','PC_div_orig',
                                'PC_dmt', 'TPC_c', 'Model'])
for mod in tqdm(['M2_bm', 'M2_top', 'M4_top', 'M_best']):
    for seas, df_sub in df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'train'].groupby('season'):
        if len(df_sub[(df_sub['PC_dmt'] == ref_pc) & (df_sub['TPC_c'] == ref_TPC)]) < 100:
            continue
        else:
            df_tmp = df_sub[(df_sub['TPC_c'] == ref_TPC) & (df_sub['PC_dmt'] == ref_pc)]
            for cp_q in cp_q_dict:
                df_pred = df_pred.append(pd.DataFrame({
                    'season': seas,
                    'Relative CP': cp_q,
                    'dFence_orig': df_tmp['dFence_orig'].median(),
                    'dTank_orig': df_tmp['dTank_orig'].median(),
                    'Biomass_orig': np.arange(df_tmp['Biomass_orig'][np.abs(df_tmp['CP_orig'] - df_tmp['CP_orig'].quantile(cp_q_dict[cp_q])) < 0.25].min(), 
                         df_tmp['Biomass_orig'][np.abs(df_tmp['CP_orig'] - df_tmp['CP_orig'].quantile(cp_q_dict[cp_q])) < 0.25].max(),  10),
                    'CP_orig': df_tmp['CP_orig'].quantile(cp_q_dict[cp_q]),
                    'PC_div_orig': df_tmp['PC_div_orig'].median(),
                    'PC_dmt': ref_pc,
                    'TPC_c': ref_TPC,
                    'Model': mod
                }))
        for c in ['dFence', 'dTank', 'Biomass', 'CP', 'PC_div']:
            scaler = scaler_dict[seas][c]
            df_pred.loc[df_pred['season'] == seas, c] = scaler.transform(
                df_pred.loc[df_pred['season'] == seas, c + '_orig'].values.reshape(-1, 1)).flatten()
        df_pred.loc[(df_pred['season'] == seas) & (df_pred['Model'] == mod), 
                    'mod_pred'] = seas_mod_dict[seas][mod].predict(df_pred.loc[(df_pred['season'] == seas) & (df_pred['Model'] == mod)])
df_pred['season'] = pd.Categorical(df_pred['season'], ['early', 'mid', 'late'])
df_pred['pred_rel_freq'] = np.nan
for seas in df_pred.season.unique():
    df_wkly_grid_sub = df_wkly_grid_full[(df_wkly_grid_full['season'] == seas) & (df_wkly_grid_full['mod_data'] == 'train')]
    df_pred.loc[df_pred['season'] == seas, 'pred_rel_freq'] = (df_pred.loc[df_pred['season'] == seas, 'mod_pred'] * df_wkly_grid_sub['grazing_wkly_sum'].mean()) / (df_wkly_grid_sub['grazing_wkly_sum'].mean() / df_wkly_grid_sub.groupby(
        ['week', 'Pasture', 'Steer_ID'])['grazing_wkly_sum'].count().mean())

  0%|          | 0/4 [00:00<?, ?it/s]

In [128]:
"""
Plot marginal effects of biomass and relative CP by model and season
"""
x_var = 'Biomass_orig'
g = sns.FacetGrid(df_pred[df_pred['Model'] == 'M_best'], col='season', hue='Relative CP', col_order=['early', 'mid', 'late'], 
                  legend_out=True, sharey=False, sharex=True, palette='viridis')
#plt.figure()
g.map_dataframe(sns.lineplot, x=x_var, y='pred_rel_freq').add_legend()
g.set_xlabels('Biomass (kg/ha)')
g.set_ylabels('Relative probability of selection')
axes = g.fig.axes
for ax in axes:
    ax.axhline(y=1.0, linestyle='dashed', color='grey')
    ax.set_ylim((0.0, ax.get_ylim()[-1]))
    if ax.get_ylim()[-1] < 2.0:
        ax.set_ylim((0.0, 2.0))
    cur_title = ax.get_title()
    ax.set_title(re.sub('season = ', '', cur_title + '-season'))
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width*0.9, box.height*0.9])
#g.fig.suptitle('Marginal effects of Biomass and Relative CP')
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/Biomass_X_relCP.png', 
            bbox_inches='tight', dpi=300)
#plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [23]:
"""
Compare top model by plant community diversity: how does selection change with plant community diversity within dominant plant communities?
"""
ref_TPC = 'Flat Plains'
df_pred = pd.DataFrame(columns=['season','dFence_orig', 'dTank_orig', 'Biomass_orig', 'CP_orig','PC_div_orig',
                                'PC_dmt', 'TPC_c', 'Model'])
for ref_pc in df_wkly_grid_full['PC_dmt'][df_wkly_grid_full['mod_data'] == 'train'].unique():
    print(ref_pc)
    for mod in tqdm(['M_best']):
        for seas, df_sub in df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'train'].groupby('season'):
            if len(df_sub[(df_sub['PC_dmt'] == ref_pc) & (df_sub['TPC_c'] == ref_TPC)]) < 100:
                continue
            else:
                df_tmp = df_sub[(df_sub['TPC_c'] == ref_TPC) & (df_sub['PC_dmt'] == ref_pc)]
                df_pred = df_pred.append(pd.DataFrame({
                    'season': seas,
                    'dFence_orig': df_tmp['dFence_orig'].median(),
                    'dTank_orig': df_tmp['dTank_orig'].median(),
                    'Biomass_orig': df_tmp['Biomass_orig'].median(),
                    'CP_orig': df_tmp['CP_orig'].quantile(cp_q_dict[cp_q]),
                    'PC_div_orig': np.arange(df_tmp['PC_div_orig'].min(), df_tmp['PC_div_orig'].max(),  0.01),
                    'PC_dmt': ref_pc,
                    'TPC_c': ref_TPC,
                    'Model': mod
                }))
            for c in ['dFence', 'dTank', 'Biomass', 'CP', 'PC_div']:
                scaler = scaler_dict[seas][c]
                df_pred.loc[df_pred['season'] == seas, c] = scaler.transform(
                    df_pred.loc[df_pred['season'] == seas, c + '_orig'].values.reshape(-1, 1)).flatten()
            df_pred.loc[(df_pred['season'] == seas) & (df_pred['Model'] == mod), 
                        'mod_pred'] = seas_mod_dict[seas][mod].predict(df_pred.loc[(df_pred['season'] == seas) & (df_pred['Model'] == mod)])
df_pred['season'] = pd.Categorical(df_pred['season'], ['early', 'mid', 'late'])
df_pred['pred_rel_freq'] = np.nan
for seas in df_pred.season.unique():
    df_wkly_grid_sub = df_wkly_grid_full[(df_wkly_grid_full['season'] == seas) & (df_wkly_grid_full['mod_data'] == 'train')]
    df_pred.loc[df_pred['season'] == seas, 'pred_rel_freq'] = (df_pred.loc[df_pred['season'] == seas, 'mod_pred'] * df_wkly_grid_sub['grazing_wkly_sum'].mean()) / (df_wkly_grid_sub['grazing_wkly_sum'].mean() / df_wkly_grid_sub.groupby(
        ['week', 'Pasture', 'Steer_ID'])['grazing_wkly_sum'].count().mean())

C3_C4_mix


  0%|          | 0/1 [00:00<?, ?it/s]

C4


  0%|          | 0/1 [00:00<?, ?it/s]

Other


  0%|          | 0/1 [00:00<?, ?it/s]

C3


  0%|          | 0/1 [00:00<?, ?it/s]

Saltgrass


  0%|          | 0/1 [00:00<?, ?it/s]

In [24]:
"""
Plot marginal effects of plant community diversity by dominant plant community and season
"""
x_var = 'PC_div_orig'
g = sns.FacetGrid(df_pred[df_pred['Model'] == 'M_best'], col='season', hue='PC_dmt',
                  col_order=['early', 'mid', 'late'], 
                  hue_order=['C4', 'C3_C4_mix', 'C3', 'Saltgrass', 'Other'],
                  legend_out=True, sharey=False)
#plt.figure()
g.map_dataframe(sns.lineplot, x=x_var, y='pred_rel_freq').add_legend()
g.set_xlabels('Diversity (Shannon H)')
g.set_ylabels('Relative probability of selection')
axes = g.axes.flat
for ax in axes:
    ax.axhline(y=1.0, linestyle='dashed', color='grey')
    ax.set_ylim((0.0, ax.get_ylim()[-1]))
    if ax.get_ylim()[-1] < 2.0:
        ax.set_ylim((0.0, 2.0))
    cur_title = ax.get_title()
    ax.set_title(re.sub('season = ', '', cur_title + '-season'))
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width*0.85, box.height*0.9])
leg = g.legend
# change legend texts
new_title = 'Plant community'
leg.set_title(new_title, prop={'weight': 'bold',
        'size': 12,
        })
new_labels=['C4', 'C3/C4 mix', 'C3', 'Saltgrass', 'Other']
for t, l in zip(leg.texts, new_labels):
    t.set_text(l)
g.fig.suptitle('Marginal effects of Plant Community Diversity by Plant Community and Season')
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/PC_div_X_PC_dmt.png',
            bbox_inches='tight', dpi=300)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [25]:
"""
Compare models by season: how does selection change with distance to fence and watertank
"""
ref_TPC = 'Flat Plains'
ref_pc = 'C3_C4_mix'
df_pred = pd.DataFrame(columns=['season','dFence_orig', 'dTank_orig', 'Biomass_orig', 'CP_orig','PC_div_orig',
                                'PC_dmt', 'TPC_c', 'Feature', 'Model'])
for mod in tqdm(['M1', 'M_best']):
    for seas, df_sub in df_wkly_grid_full[df_wkly_grid_full['mod_data'] == 'train'].groupby('season'):
        if len(df_sub[(df_sub['PC_dmt'] == ref_pc) & (df_sub['TPC_c'] == ref_TPC)]) < 100:
            continue
        else:
            df_tmp = df_sub[(df_sub['TPC_c'] == ref_TPC) & (df_sub['PC_dmt'] == ref_pc)]
            for feat in ['Fence', 'Tank']:
                if feat == 'Fence':
                    dFence = np.arange(df_tmp['dFence_orig'].min(), df_tmp['dFence_orig'].max(), 10)
                else:
                    dFence = df_tmp['dFence_orig'].median()
                if feat == 'Tank':
                    dTank = np.arange(df_tmp['dTank_orig'].min(), df_tmp['dTank_orig'].max(), 10)
                else:
                    dTank = df_tmp['dTank_orig'].median()
                    
                df_pred = df_pred.append(pd.DataFrame({
                    'season': seas,
                    'dFence_orig': dFence,
                    'dTank_orig': dTank,
                    'Biomass_orig': df_tmp['Biomass_orig'].median(),
                    'CP_orig': df_tmp['CP_orig'].median(),
                    'PC_div_orig': df_tmp['PC_div_orig'].median(),
                    'PC_dmt': ref_pc,
                    'TPC_c': ref_TPC,
                    'Feature': feat,
                    'Model': mod
                }))
        for c in ['dFence', 'dTank', 'Biomass', 'CP', 'PC_div']:
            scaler = scaler_dict[seas][c]
            df_pred.loc[df_pred['season'] == seas, c] = scaler.transform(
                df_pred.loc[df_pred['season'] == seas, c + '_orig'].values.reshape(-1, 1)).flatten()
        df_pred.loc[(df_pred['season'] == seas) & (df_pred['Model'] == mod), 
                    'mod_pred'] = seas_mod_dict[seas][mod].predict(df_pred.loc[(df_pred['season'] == seas) & (df_pred['Model'] == mod)])
df_pred['season'] = pd.Categorical(df_pred['season'], ['early', 'mid', 'late'])
df_pred['pred_rel_freq'] = np.nan
for seas in df_pred.season.unique():
    df_wkly_grid_sub = df_wkly_grid_full[(df_wkly_grid_full['season'] == seas) & (df_wkly_grid_full['mod_data'] == 'train')]
    df_pred.loc[df_pred['season'] == seas, 'pred_rel_freq'] = (df_pred.loc[df_pred['season'] == seas, 'mod_pred'] * df_wkly_grid_sub['grazing_wkly_sum'].mean()) / (df_wkly_grid_sub['grazing_wkly_sum'].mean() / df_wkly_grid_sub.groupby(
        ['week', 'Pasture', 'Steer_ID'])['grazing_wkly_sum'].count().mean())

  0%|          | 0/2 [00:00<?, ?it/s]

In [26]:
df_pred['dFence_orig'][df_pred['Feature'] == 'Tank'] = np.nan
df_pred['dTank_orig'][df_pred['Feature'] == 'Fence'] = np.nan
df_pred_long = pd.melt(df_pred, id_vars=['season', 'Model', 'Feature', 'pred_rel_freq'], value_vars=['dFence_orig', 'dTank_orig'], value_name='Distance')

C:\Users\SEAN~1.KEA\AppData\Local\Temp/ipykernel_6360/1498154917.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['dFence_orig'][df_pred['Feature'] == 'Tank'] = np.nan
C:\Users\SEAN~1.KEA\AppData\Local\Temp/ipykernel_6360/1498154917.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_pred['dTank_orig'][df_pred['Feature'] == 'Fence'] = np.nan


In [28]:
"""
Plot marginal effects of distance to fence by model and season
"""
x_var = 'Distance'
g = sns.FacetGrid(df_pred_long[(df_pred_long['Model'].isin(['M1', 'M_best'])) & (df_pred_long['Feature'] == 'Fence')],
                  col='season', hue='Model',
                               col_order=['early', 'mid', 'late'], 
                  legend_out=True, sharey='col', sharex=True)
#plt.figure()
g.map_dataframe(sns.lineplot, x=x_var, y='pred_rel_freq').add_legend(bbox_to_anchor=(0.5, 0.98), 
                                                                     loc='upper center',
                                                                                                   borderaxespad=0, 
                                                                                                   ncol=2, 
                                                                                                   frameon=True)
g.set_xlabels('Distance to nearest fence (m)')
g.set_ylabels('Relative probability of selection')
axes = g.fig.axes
for ax in axes:
    ax.axhline(y=1.0, linestyle='dashed', color='grey')
    ax.set_ylim((0.0, ax.get_ylim()[-1]))
    if ax.get_ylim()[-1] < 2.0:
        ax.set_ylim((0.0, 2.0))
    cur_title = ax.get_title()
    ax.set_title(re.sub('season = ', '', cur_title + '-season'))
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width, box.height*0.8])
leg = g.legend
# change legend texts
new_title = 'Model type'
leg.set_title(new_title, prop={'weight': 'bold',
        'size': 12,
        })
new_labels = ['Null model (M0)', 'Fully paramaterized (M5)']
for t, l in zip(leg.texts, new_labels):
    t.set_text(l)
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/dfence.png', 
            bbox_inches='tight', dpi=300)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
"""
Plot marginal effects of distance to water tank by model and season
"""
x_var = 'Distance'
g = sns.FacetGrid(df_pred_long[(df_pred_long['Model'].isin(['M1', 'M_best'])) & (df_pred_long['Feature'] == 'Tank')],
                  col='season', hue='Model',
                               col_order=['early', 'mid', 'late'], 
                  legend_out=True, sharey='col', sharex=True)
#plt.figure()
g.map_dataframe(sns.lineplot, x=x_var, y='pred_rel_freq').add_legend(bbox_to_anchor=(0.5, 0.98), 
                                                                     loc='upper center',
                                                                                                   borderaxespad=0, 
                                                                                                   ncol=2, 
                                                                                                   frameon=True)
g.set_xlabels('Distance to water tank (m)')
g.set_ylabels('Relative probability of selection')
axes = g.fig.axes
for ax in axes:
    ax.axhline(y=1.0, linestyle='dashed', color='grey')
    ax.set_ylim((0.0, ax.get_ylim()[-1]))
    if ax.get_ylim()[-1] < 2.0:
        ax.set_ylim((0.0, 2.0))
    cur_title = ax.get_title()
    ax.set_title(re.sub('season = ', '', cur_title + '-season'))
    box = ax.get_position()
    ax.set_position([box.x0, box.y0, box.width, box.height*0.8])
leg = g.legend
# change legend texts
new_title = 'Model type'
leg.set_title(new_title, prop={'weight': 'bold',
        'size': 12,
        })
new_labels = ['Null model (M0)', 'Fully paramaterized (M5)']
for t, l in zip(leg.texts, new_labels):
    t.set_text(l)
plt.savefig('C:/Users/sean.kearney/OneDrive - USDA/Documents/Projects/GPS_v_hetgen/figures/dtank.png', 
            bbox_inches='tight', dpi=300)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [32]:
plt.figure()
sns.kdeplot(x='grazing_rel_freq', hue='season', 
            data=df_wkly_grid_train, bw_adjust=2.0, clip=(0, None),
           common_norm=False, cumulative=False, common_grid=False)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='grazing_rel_freq', ylabel='Density'>

In [49]:
df_wkly_grid_train['grazing_rel_freq_seas'] = (df_wkly_grid_train.groupby(['season', 'Pasture', 'Steer_ID', 'UTM_X', 'UTM_Y']).grazing_secs.transform('sum') /\
(df_wkly_grid_train.groupby(['season', 'Pasture', 'Steer_ID', 'UTM_X', 'UTM_Y']).grazing_wkly_sum.transform('sum') /\
 df_wkly_grid_train.groupby(['week', 'season', 'Pasture', 'Steer_ID']).grazing_secs.transform(lambda x: np.mean(len(x))))).reset_index(drop=True)

In [50]:
#df_wkly_grid_train[df_wkly_grid_train['mod_data'] == 'train']

In [51]:
#df_wkly_grid_train[(df_wkly_grid_train.Pasture == '7NW') &
#                 (df_wkly_grid_train.UTM_X == 524055.0) & 
#                 (df_wkly_grid_train.UTM_Y == 4522695.0) &
                  #(df_wkly_grid_train.Steer_ID == '2287_D1_2017') & 
#                 (df_wkly_grid_train.mod_data == 'train')].sort_values('Steer_ID')

In [52]:
df_wkly_grid_train['grazing_rel_freq_seas_grp'] = pd.cut(df_wkly_grid_train.groupby(['season', 'Pasture', 'UTM_X', 'UTM_Y'])['grazing_rel_freq_seas'].transform('mean'), 
                                          bins=rel_freq_bins, 
                                          labels=bin_labs,
                                          include_lowest=True)
df_wkly_grid_train['grazing_rel_freq_past_grp'] = pd.cut(df_wkly_grid_train.groupby(['week', 'Pasture', 'UTM_X', 'UTM_Y'])['grazing_rel_freq'].transform('mean'), 
                                          bins=rel_freq_bins, 
                                          labels=bin_labs,
                                          include_lowest=True)

In [59]:
df_wkly_grid_train.groupby('season')['grazing_secs'].apply(lambda x: np.sum(np.isnan(x)))

season
early    1
mid      0
late     0
Name: grazing_secs, dtype: int64

In [40]:
df_seas_grid = df_wkly_grid_train.groupby(['season', 'mod_data', 'Pasture',
                           'UTM_X', 'UTM_Y'])[['grazing_rel_freq_seas', 
                                                           'grazing_rel_freq_seas_grp',
                                                           'Biomass_orig',
                                                           'CP_orig', 
                                                           'PC_dmt', 
                                                           'PC_div_orig']].aggregate(func={
    'grazing_rel_freq_seas': 'last',
    'grazing_rel_freq_seas_grp': 'last',
    'Biomass_orig': np.mean,
    'CP_orig': np.mean,
    'PC_dmt': 'last',
    'PC_div_orig': 'last'
}).reset_index()
    #['mod_data', 'Pasture', 'Steer_ID', 'UTM_X', 'UTM_Y'])['grazing_rel_freq_wkly_grp'].transform(lambda x: '_'.join(x.astype('str')))

In [45]:
df_seas_grid['grazing_rel_freq_seas_grp'].dropna()

691397      Low
691398      Low
691399      Low
691400     Avg.
691401      Low
           ... 
4128583    Avg.
4128584      Hi
4128585      Hi
4128586      Hi
4128904      Hi
Name: grazing_rel_freq_seas_grp, Length: 25953, dtype: category
Categories (5, object): ['Low' < 'Mod. Low' < 'Avg.' < 'Mod. Hi' < 'Hi']

In [41]:
df_seas_grid = df_seas_grid[df_seas_grid['mod_data'] == 'train']
df_seas_grid['grazing_rel_freq_seas_traj'] = df_seas_grid.groupby(['mod_data', 'Pasture', 
                      'UTM_X', 'UTM_Y'])['grazing_rel_freq_seas_grp'].transform('_'.join)
df_seas_grid['season'] = pd.Categorical(df_seas_grid['season'], ['early', 'mid', 'late'])

TypeError: sequence item 0: expected str instance, float found

In [86]:
plt.figure()
sns.pointplot(x='season', y='Biomass', hue='grazing_rel_freq_seas_traj', 
            data=df_seas_grid[df_seas_grid['grazing_rel_freq_seas_traj'].isin(['Low_Low_Low',
                                                                               'Hi_Hi_Hi'])])

C:\Users\SEAN~1.KEA\AppData\Local\Temp/ipykernel_96040/2580292971.py:1: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  plt.figure()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='season', ylabel='Biomass'>

In [89]:
plt.figure()
sns.pointplot(x='season', y='PC_div', hue='grazing_rel_freq_seas_traj', 
            data=df_seas_grid[df_seas_grid['grazing_rel_freq_seas_traj'].isin(['Low_Low_Low',
                                                                               'Hi_Hi_Hi'])])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='season', ylabel='PC_div'>

In [53]:
df_seas_grid['grazing_rel_freq_seas_traj'][df_seas_grid['season'] == 'late'].value_counts()

Low_Low_Low                 733
Hi_Hi_Hi                    296
Mod. Low_Low_Low            276
Low_Mod. Low_Low            245
Low_Low_Mod. Low            171
                           ... 
Mod. Hi_Mod. Hi_Mod. Hi      15
Mod. Hi_Mod. Hi_Mod. Low     13
Mod. Low_Mod. Hi_Mod. Hi     12
Mod. Hi_Mod. Low_Mod. Hi     11
Low_Mod. Hi_Mod. Hi           8
Name: grazing_rel_freq_seas_traj, Length: 125, dtype: int64

In [65]:
#plt.figure()
df_plot = df_wkly_grid_train[df_wkly_grid_train['mod_data'] == 'train'].groupby(
    ['season', 'week', 'Pasture', 'Steer_ID'])['grazing_rel_freq_seas_grp'].value_counts().reset_index().groupby(
    ['season', 'Pasture', 'level_4']).mean().reset_index().pivot(columns='level_4',
                                                               index=['season', 'Pasture'], values='grazing_rel_freq_seas_grp').reset_index()#.plot(kind='bar', stacked=True)
df_plot['pct_Hi'] = df_plot[['Hi']].sum(axis=1) / df_plot[['Low', 'Mod. Low', 'Avg.', 
                                                  'Mod. Hi', 'Hi']].sum(axis=1)
df_plot['season'] = pd.Categorical(df_plot['season'], ['early', 'mid', 'late'])
#df_plot
plt.figure()
sns.pointplot(x='season', y='pct_Hi', hue='Pasture', data=df_plot)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

<AxesSubplot:xlabel='season', ylabel='pct_Hi'>

In [66]:
df_wkly_grid_train[df_wkly_grid_train['mod_data'] == 'train'].pivot(index=['Pasture', 'UTM_X', 'UTM_Y'])

level_4,season,Pasture,Low,Mod. Low,Avg.,Mod. Hi,Hi,pct_Hi
0,early,15E,524.833333,253.000000,208.666667,122.166667,318.833333,0.223351
1,early,17N,393.833333,342.000000,261.000000,97.666667,288.333333,0.208509
2,early,20SE,422.000000,294.000000,315.333333,119.666667,269.000000,0.189437
3,early,26E,415.333333,313.000000,272.000000,125.333333,249.333333,0.181333
4,early,31E,402.666667,355.333333,360.666667,179.333333,323.666667,0.199589
5,early,7NW,392.833333,318.000000,270.500000,140.833333,264.500000,0.190745
6,late,15E,354.250000,246.250000,368.500000,166.750000,295.000000,0.206186
7,late,17N,486.750000,249.750000,266.000000,97.250000,283.000000,0.204665
8,late,20SE,424.750000,291.250000,287.250000,141.250000,276.250000,0.194440
9,late,26E,450.500000,251.500000,264.000000,116.500000,295.000000,0.214156


In [ ]:
#plt.figure()
df_plot = df_wkly_grid_train[df_wkly_grid_train['mod_data'] == 'train'].groupby(
    ['season', 'week', 'Pasture', 'Steer_ID'])['grazing_rel_freq_seas_grp'].value_counts().reset_index().groupby(
    ['season', 'Pasture', 'level_4']).mean().reset_index().pivot(columns='level_4',
                                                               index=['season', 'Pasture'], values='grazing_rel_freq_seas_grp').reset_index()#.plot(kind='bar', stacked=True)
df_plot['pct_Hi'] = df_plot[['Hi']].sum(axis=1) / df_plot[['Low', 'Mod. Low', 'Avg.', 
                                                  'Mod. Hi', 'Hi']].sum(axis=1)
df_plot['season'] = pd.Categorical(df_plot['season'], ['early', 'mid', 'late'])
#df_plot
plt.figure()
sns.pointplot(x='season', y='pct_Hi', hue='Pasture', data=df_plot)

In [46]:
"""
Save data to make Sankey diagrams in R
"""
df_sankey = pd.DataFrame(columns=['class_early', 'class_mid', 'class_late', 'value'])
df_sankey['class_early'] = df_seas_grid[df_seas_grid['season'] == 'early']['grazing_rel_freq_seas_grp'].reset_index(drop=True)
df_sankey['class_mid'] = df_seas_grid[df_seas_grid['season'] == 'mid']['grazing_rel_freq_seas_grp'].reset_index(drop=True)
df_sankey['class_late'] = df_seas_grid[df_seas_grid['season'] == 'late']['grazing_rel_freq_seas_grp'].reset_index(drop=True)
df_sankey['value'] = 1
sankey_grp = df_sankey.groupby(['class_early', 'class_mid', 'class_late']).sum().reset_index(drop=False)
sankey_grp = sankey_grp.reindex(columns=sankey_grp.columns.tolist())
sankey_grp.to_csv('C:/SPK_local/zTEMP/gps_grid_sankey2.csv')

In [11]:
aicw_func = lambda x: np.exp(-0.5 * (x - x.min())) / np.sum(np.exp(-0.5 * (x - x.min())))
aicd_func = lambda x: x - x.min()

In [38]:
"""
Analyze best results for specific model comparisons defined in dictionary above
"""
for k in mod_aic_dict:
    df_results.loc[df_results['model'].isin(mod_aic_dict[k]), 
                   'AICw_' + k] = df_results.loc[df_results['model'].isin(mod_aic_dict[k])].groupby(
        ['week', 'Pasture'])['AIC'].transform(aicw_func)
    df_results.loc[df_results['model'].isin(mod_aic_dict[k]), 
                   'AICd_' + k] = df_results.loc[df_results['model'].isin(mod_aic_dict[k])].groupby(
        ['week', 'Pasture'])['AIC'].transform(aicd_func)
    df_results['top_model_' + k] = df_results.groupby(['week', 'Pasture'])['AICw_' + k].transform(
        lambda x: [int(i) for i in x == x.max()])
    df_results['candidate_model_' + k] = df_results.groupby(['week', 'Pasture'])['AICd_' + k].transform(
        lambda x: [int(i) for i in x <= 2.0])

In [39]:
"""
Display the results for the desired model comparison
"""
mod_grp = 'Hw2'
display(df_results.loc[df_results['model'].isin(mod_aic_dict[mod_grp])].groupby(['season', 'model'])[['top_model_' + mod_grp, 'candidate_model_' + mod_grp,
                                                                                'AICw_' + mod_grp, 'AICd_' + mod_grp]].aggregate(
    func={'AICd_' + mod_grp: [np.median, np.std], 'AICw_' + mod_grp: [np.mean, np.std], 
         'top_model_' + mod_grp: np.sum, 'candidate_model_' + mod_grp: np.sum}, axis=1).sort_values(['season', ('AICd_' + mod_grp, 'median')]))
display(df_results.loc[df_results['model'].isin(mod_aic_dict[mod_grp])].groupby(['model'])[['top_model_' + mod_grp, 'candidate_model_' + mod_grp, 
                                                                      'AICw_' + mod_grp, 'AICd_' + mod_grp]].aggregate(
    func={'AICd_' + mod_grp: [np.median, np.std], 'AICw_' + mod_grp: [np.mean, np.std], 
          'top_model_' + mod_grp: np.sum, 'candidate_model_' + mod_grp: np.sum}, axis=1).sort_values(('AICd_' + mod_grp, 'median')))

AICd_Hw2                  AICw_Hw2           top_model_Hw2  \
               median         std          mean       std           sum   
season model                                                              
early  M2g      0.275    1.081198  5.061748e-01  0.296941             9   
       M2f      3.650   15.116630  2.215112e-01  0.240397             4   
       M2e      6.540   15.016862  1.527321e-01  0.249046             3   
       M2d     19.350   53.099368  3.105445e-02  0.063824             0   
       M2b     28.545   41.595856  4.482712e-02  0.091801             1   
       M2a     46.480   59.347148  2.105054e-02  0.066307             0   
       M2c     55.240   83.766058  2.264990e-02  0.095805             1   
late   M2g      0.000    1.393833  5.669245e-01  0.357838             7   
       M2f      3.970   13.607834  1.704526e-01  0.196187             2   
       M2e     17.635   17.977953  1.075712e-01  0.192545             1   
       M2d     22.035   29.178889  6.553204e-02  0.153503             1   
       M2b     28.425   32.462956  8.392830e-02  0.201161             1   
       M2c     34.785  163.188775  6.374583e-04  0.001683             0   
       M2a     48.850   38.307737  4.953916e-03  0.011591             0   
mid    M2g      0.000    1.147310  6.540285e-01  0.360957            12   
       M2f     10.395   34.692182  1.217726e-01  0.168370             2   
       M2e     16.580   29.966809  1.357531e-01  0.215604             2   
       M2d     35.230   36.685161  5.904228e-02  0.134963             1   
       M2b     90.820  106.598850  2.932907e-02  0.123558             1   
       M2a    105.395  110.054974  7.381332e-05  0.000313             0   
       M2c    160.450  114.654101  5.493250e-07  0.000002             0   

             candidate_model_Hw2  
                             sum  
season model                      
early  M2g                    16  
       M2f                     8  
       M2e                     5  
       M2d                     3  
       M2b                     2  
       M2a                     1  
       M2c                     1  
late   M2g                    10  
       M2f                     5  
       M2e                     3  
       M2d                     2  
       M2b                     2  
       M2c                     0  
       M2a                     0  
mid    M2g                    16  
       M2f                     6  
       M2e                     5  
       M2d                     2  
       M2b                     1  
       M2a                     0  
       M2c                     0

AICd_Hw2              AICw_Hw2           top_model_Hw2  \
        median         std      mean       std           sum   
model                                                          
M2g      0.000    1.166435  0.576807  0.336463            28   
M2f      4.550   25.114508  0.171345  0.205110             8   
M2e     13.790   22.678330  0.135075  0.219630             6   
M2d     20.395   41.549821  0.050169  0.117503             2   
M2b     38.480   83.472705  0.048791  0.136041             3   
M2a     71.340   89.428555  0.009160  0.041377             0   
M2c     87.710  120.894697  0.008653  0.058657             1   

      candidate_model_Hw2  
                      sum  
model                      
M2g                    42  
M2f                    19  
M2e                    13  
M2d                     7  
M2b                     5  
M2a                     1  
M2c                     1